In [1]:
# Thanks to Nicks Niche for the starter code
# in case needed
#!pip install pulp

In [2]:
# Considerations
# - Players are not independent, how to account for stacks, correlations?
# - We are playing against others and have to share the winnings, how to get slight edge?
# -- maybe cross off top picks by position
# -- Limit total salary (eg everyone trying to spend 50k, maybe spend 48)
# - We are passing the mean score, but need to consider variance. Might want more risk to get ahead.
# -- Could vary the projections by x%

In [3]:
import pandas as pd
import numpy as np
from pulp import *

from src.utils import fix_name_mismatches
from src.components.import_data import ImportData
from src.components.optimizer import OptimizeLineup

### Set Inputs

In [4]:
week = 15
game_mode = 'showdown' # 'showdown' or 'classic'
rank_src = 'adhoc' # 'adhoc' or 'FP'
matchup = 'LAC_LV'

adhoc_pt_col = 'Points' # 'Floor', 'Points', or 'Ceiling'

# to add uncertainty to point projections
variability_pctg = 15

n_lineups = 25
add_results = False # To review lineups along with actual outcomes

# scratch players
scratches = ['Keenan Allen', 'Josh Jacobs']

# Add must haves in lineup - if forcing defense put space at end (ugh)
force_ins = []


### Read in Draft Kings Players/Salaries and Rankings

In [5]:
gd = ImportData(week, game_mode, rank_src, matchup, add_results)
players = gd.read_dk()
player_rankings = gd.read_rankings(players, adhoc_pt_col=adhoc_pt_col)

In [6]:
# QA check for any large names missing rank data
# May signal a name mismatch across sources
mask = player_rankings['FPPG'].isnull() #& player_rankings['Salary'] > 500
player_rankings[mask].sort_values('Salary', ascending=False).head(5)

,Id,Position,Salary,FPPG,ActualFP
0,Justin Herbert,CPT,17100,NaN,NaN
8,Justin Herbert,FLEX,11400,NaN,NaN
17,Jimmy Garoppolo,CPT,9000,NaN,NaN
18,Brian Hoyer,CPT,9000,NaN,NaN
19,Max Duggan,CPT,9000,NaN,NaN


In [7]:
availables = player_rankings.groupby(['Position', 'Id', 'FPPG', 'Salary']).agg('count')
availables = availables.reset_index()

# screening on availables
#mask = availables['Salary'] > 900
#availables = availables[mask]

availables = availables[~availables['Id'].isin(scratches)].reset_index(drop=True)

#check for any force ins not in list
print(f"Names Not Found: {list(set(force_ins) - set(availables['Id']))}")

Names Not Found: []


In [8]:
availables

,Position,Id,FPPG,Salary,ActualFP
0,CPT,Aidan O'Connell,19.200,14400,0
1,CPT,Ameer Abdullah,4.665,7200,0
2,CPT,Austin Ekeler,19.050,16500,0
3,CPT,Cameron Dicker,10.500,7500,0
4,CPT,Chargers,7.800,6600,0
5,CPT,Daniel Carlson,12.210,6900,0
6,CPT,Davante Adams,17.400,16800,0
7,CPT,Easton Stick,19.800,14100,0
8,CPT,Gerald Everett,8.145,7800,0
9,CPT,Jakobi Meyers,13.185,12900,0


### Run Optimizer

In [11]:
op = OptimizeLineup(game_mode, availables, n_lineups, force_ins, add_results, variability_pctg)
results = op.run_optimizer()
results.head(20).T

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,CPT_Austin_Ekeler,CPT_Austin_Ekeler,CPT_Easton_Stick,CPT_Easton_Stick,CPT_Austin_Ekeler,CPT_Aidan_O'Connell,CPT_Austin_Ekeler,CPT_Austin_Ekeler,CPT_Easton_Stick,CPT_Zamir_White,CPT_Aidan_O'Connell,CPT_Zamir_White,CPT_Aidan_O'Connell,CPT_Easton_Stick,CPT_Joshua_Palmer,CPT_Aidan_O'Connell,CPT_Davante_Adams,CPT_Easton_Stick,CPT_Daniel_Carlson,CPT_Daniel_Carlson
2,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Austin_Ekeler,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Austin_Ekeler,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Austin_Ekeler,FLEX_Austin_Ekeler,FLEX_Cameron_Dicker,FLEX_Aidan_O'Connell,FLEX_Aidan_O'Connell,FLEX_Cameron_Dicker,FLEX_Austin_Ekeler,FLEX_Aidan_O'Connell,FLEX_Austin_Ekeler,FLEX_Aidan_O'Connell
3,FLEX_Cameron_Dicker,FLEX_Daniel_Carlson,FLEX_Chargers_,FLEX_Cameron_Dicker,FLEX_Ameer_Abdullah,FLEX_Chargers_,FLEX_Chargers_,FLEX_Cameron_Dicker,FLEX_Ameer_Abdullah,FLEX_Austin_Ekeler,FLEX_Easton_Stick,FLEX_Cameron_Dicker,FLEX_Easton_Stick,FLEX_Austin_Ekeler,FLEX_Austin_Ekeler,FLEX_Daniel_Carlson,FLEX_Cameron_Dicker,FLEX_Austin_Ekeler,FLEX_Davante_Adams,FLEX_Austin_Ekeler
4,FLEX_Chargers_,FLEX_Easton_Stick,FLEX_Daniel_Carlson,FLEX_Daniel_Carlson,FLEX_Cameron_Dicker,FLEX_Daniel_Carlson,FLEX_Daniel_Carlson,FLEX_Daniel_Carlson,FLEX_Daniel_Carlson,FLEX_Chargers_,FLEX_Joshua_Palmer,FLEX_Davante_Adams,FLEX_Jakobi_Meyers,FLEX_Daniel_Carlson,FLEX_Daniel_Carlson,FLEX_Davante_Adams,FLEX_Daniel_Carlson,FLEX_Chargers_,FLEX_Easton_Stick,FLEX_Easton_Stick
5,FLEX_Daniel_Carlson,FLEX_Joshua_Palmer,FLEX_Davante_Adams,FLEX_Davante_Adams,FLEX_Daniel_Carlson,FLEX_Easton_Stick,FLEX_Easton_Stick,FLEX_Easton_Stick,FLEX_Davante_Adams,FLEX_Davante_Adams,FLEX_Raiders_,FLEX_Easton_Stick,FLEX_Joshua_Palmer,FLEX_Gerald_Everett,FLEX_Easton_Stick,FLEX_Easton_Stick,FLEX_Easton_Stick,FLEX_Daniel_Carlson,FLEX_Gerald_Everett,FLEX_Jakobi_Meyers
6,FLEX_Easton_Stick,FLEX_Zamir_White,FLEX_Raiders_,FLEX_Raiders_,FLEX_Easton_Stick,FLEX_Joshua_Palmer,FLEX_Michael_Mayer,FLEX_Raiders_,FLEX_Raiders_,FLEX_Jakobi_Meyers,FLEX_Zamir_White,FLEX_Jakobi_Meyers,FLEX_Zamir_White,FLEX_Quentin_Johnston,FLEX_Quentin_Johnston,FLEX_Gerald_Everett,FLEX_Zamir_White,FLEX_Raiders_,FLEX_Joshua_Palmer,FLEX_Raiders_
ProjFP,69.627649,67.670831,67.62298,67.506531,67.48608,67.448151,67.404761,65.349759,65.331863,65.257163,65.243409,65.220881,65.210436,65.168466,65.105147,65.053045,65.037563,65.023541,64.939897,64.825343


In [10]:
# Current entry is below
results.loc[2,]

1             CPT_Easton_Stick
2         FLEX_Aidan_O'Connell
3          FLEX_Cameron_Dicker
4          FLEX_Daniel_Carlson
5           FLEX_Jakobi_Meyers
6           FLEX_Joshua_Palmer
ProjFP                 68.3641
Name: 2, dtype: object

1          CPT_Trevor_Lawrence
2                FLEX_Bengals_
3         FLEX_Brandon_McManus
4           FLEX_Ja'Marr_Chase
5           FLEX_Jake_Browning
6               FLEX_Joe_Mixon
ProjFP               89.356369
Name: 2, dtype: object